# Snowflake Worksheets 총 정리

## GCS 연결 설정

1. GCS Integration

In [ ]:
CREATE STORAGE INTEGRATION gcs_int
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'GCS'
  ENABLED = TRUE
  STORAGE_ALLOWED_LOCATIONS = ('gcs://test-devcourse');

DESC STORAGE INTEGRATION gcs_int;

CREATE STORAGE INTEGRATION gcs_proj2
  TYPE = EXTERNAL_STAGE
  STORAGE_PROVIDER = 'GCS'
  ENABLED = TRUE
  STORAGE_ALLOWED_LOCATIONS = ('gcs://programmers-devcourse-project2');

DESC STORAGE INTEGRATION gcs_proj2;

2. File Format

In [ ]:
CREATE OR REPLACE FILE format csv_format_2
TYPE = csv
FIELD_DELIMITER= '|'
SKIP_HEADER = 1
EMPTY_FIELD_AS_NULL = false;

CREATE OR REPLACE FILE format my_csv_format
TYPE = csv
FIELD_DELIMITER= ','
SKIP_HEADER = 1
NULL_IF = ('NULL', 'null')
EMPTY_FIELD_AS_NULL = true
DATE_FORMAT='YYYYMMDD';

CREATE OR REPLACE FILE format my_csv_format2
TYPE = csv
FIELD_DELIMITER= ','
SKIP_HEADER = 1
NULL_IF = ('NULL', 'null')
EMPTY_FIELD_AS_NULL = true;

CREATE OR REPLACE FILE format kor_csv_format
TYPE = csv
FIELD_DELIMITER= ','
SKIP_HEADER = 1
NULL_IF = ('NULL', 'null')
EMPTY_FIELD_AS_NULL = true
DATE_FORMAT='YYYYMMDD'
encoding = 'EUC-KR';

3. Stage

In [ ]:
create or replace stage gcs_stage_2
  url = 'gcs://test-devcourse'
  storage_integration = gcs_int
  FILE_FORMAT = csv_format_2;

create or replace stage my_gcs_stage
  url = 'gcs://test-devcourse'
  storage_integration = gcs_int
  FILE_FORMAT = my_csv_format;

list my_gcs_stage;

create or replace stage my_gcs_proj2_stage
  url = 'gcs://programmers-devcourse-project2'
  storage_integration = gcs_proj2
  FILE_FORMAT = my_csv_format;

CREATE OR REPLACE STAGE my_gcs_proj2_stage2
  url = 'gcs://programmers-devcourse-project2'
  storage_integration = gcs_proj2
  FILE_FORMAT = my_csv_format2;


## raw data 벌크 업데이트

- 서울시 데이터
    
    **SEOUL_LIFE_LOGISTICS**

In [ ]:
create or replace table Seoul_life_logistics (
    delivery_date DATE,
    sender_city VARCHAR(10),
    sender_district VARCHAR(10),
    recipient_city VARCHAR(10),
    recipient_district VARCHAR(10),
    category_furniture_and_interior INTEGER,
    category_others INTEGER,
    category_book_and_music INTEGER,
    category_digital_and_appliances INTEGER,
    category_living_and_health INTEGER,
    category_sports_and_leisure INTEGER,
    category_food INTEGER,
    category_parenting INTEGER,
    category_fashion_clothing INTEGER,
    category_fashion_accessories INTEGER,
    category_beauty_cosmetics INTEGER
);

COPY INTO raw_data.Seoul_life_logistics
FROM @my_gcs_proj2_stage/Seoul-Life-Logistics/
FILES=('SEOUL_SEOUL.csv', 'SEOUL_SIDO.csv', 'SIDO_SEOUL.csv');

- 시도 데이터
    
    **SIDO_DELIVERY_INFO**

In [ ]:
CREATE OR REPLACE TABLE Sido_delivery_info (
    receipt_date DATE,
    departure_area VARCHAR(10),
    destination_area VARCHAR(10),
    category VARCHAR(20),
    total_weight INTEGER,
    total_volume INTEGER,
    total_deliveries INTEGER
    );

COPY INTO raw_data.Sido_delivery_info
FROM(
SELECT $2, $3, $4, $5, $7, $8, $6
FROM @my_gcs_proj2_stage/Sido-Delivery-Info/
)
PATTERN='.*\.csv';

- 동 데이터
    
    ****DONG_DELIVERY_INFO_2020****
    
    ****DONG_DELIVERY_INFO_2021****
    
    ****DONG_DELIVERY_INFO_2022****

In [ ]:
create or replace table raw_data.dong_delivery_info_2022 (
    DISC_NO VARCHAR(64) primary key,
    DELIV_DATE DATE,
    RCV_DATE DATE,
    DELIV_CTPV_NM VARCHAR(50),
    DELIV_CTGG_NM VARCHAR(50),
    DELIV_ADSTRD_NM VARCHAR(50),
    EUPD_IVNM VARCHAR(150)
    );

COPY INTO raw_data.dong_delivery_info_2022(DISC_NO,DELIV_DATE,RCV_DATE ,DELIV_CTPV_NM ,DELIV_CTGG_NM ,DELIV_ADSTRD_NM,EUPD_IVNM )
  FROM (
    SELECT c.$3, DATE(c.$1, 'YYYYMMDD'), DATE(c.$17,'YYYYMMDD'), c.$20,c.$21,c.$22,c.$23
    FROM @my_gcs_stage c
  )
PATTERN = '.*22.*[0-9].*.csv'
ON_ERROR = 'continue';

- 지역별 인구수 데이터
    
    ****POPULATION_DATA_2020****
    
    ****POPULATION_DATA_2021****
    
    ****POPULATION_DATA_2022****

In [ ]:
create or replace table population_data_2020 (
    area_code varchar(20),
    area_name varchar(50),
    age_group varchar(20),
    total_population integer,
    male_population integer,
    female_population integer
);

copy into raw_data.population_data_2020
from @my_gcs_proj2_stage/Population-by-Region/Population_by_Region_Age_RegionCodes_2020.csv
FILE_FORMAT = kor_csv_format;

UPDATE raw_data.population_data_2020
SET area_name = TRIM(area_name);

- 기타 데이터
    
    **DISTANCE_BETWEEN_REGIONS**
    
    ****REGION_CODES****

In [ ]:
CREATE OR REPLACE FILE format kor_csv_format
TYPE = csv
FIELD_DELIMITER= ','
SKIP_HEADER = 1
NULL_IF = ('NULL', 'null')
EMPTY_FIELD_AS_NULL = true
DATE_FORMAT='YYYYMMDD'
encoding = 'EUC-KR';


create or replace table distance_between_regions(
    src_region varchar(10),
    dest_region varchar(10),
    distance float
);

COPY INTO raw_data.distance_between_regions
FROM @my_gcs_proj2_stage/Region_Info/distance_between_regions.csv

create or replace table region_codes(
    sido_nm varchar(32),
    sigungu_nm varchar(32),
    dong_nm varchar(32),
    region_code varchar(10)
);

COPY INTO raw_data.region_codes
FROM @my_gcs_proj2_stage/Region_Info/region_codes.csv
FILE_FORMAT = kor_csv_format;

- 기타 데이터
    
    ISO_3166_2_KR

In [ ]:
CREATE OR REPLACE TABLE ISO_3166_2_KR (
    AREA_NAME VARCHAR(50) primary key,
    ISO_3166_2_CODE VARCHAR(10)
    );

INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('강원도', 'KR-42');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('강원특별자치도', 'KR-42');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('경기도', 'KR-41');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('경상남도', 'KR-48');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('경상북도', 'KR-47');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('광주광역시', 'KR-29');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('대구광역시', 'KR-27');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('대전광역시', 'KR-30');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('부산광역시', 'KR-26');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('서울특별시', 'KR-11');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('세종특별자치시', 'KR-50');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('울산광역시', 'KR-31');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('인천광역시', 'KR-28');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('전라남도', 'KR-46');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('전라북도', 'KR-45');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('제주특별자치도', 'KR-49');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('충청남도', 'KR-44');
INSERT INTO ISO_3166_2_KR (AREA_NAME, ISO_3166_2_CODE) VALUES ('충청북도', 'KR-43');

## analytics 테이블 작성

### 지역별 물동량

*AREA_TOTAL_SUMMARY*

In [ ]:
CREATE OR REPLACE TABLE ANALYTICS.AREA_TOTAL_SUMMARY AS
SELECT
    RECEIPT_DATE,
    DEPARTURE_AREA,
    DESTINATION_AREA,
    SUM(TOTAL_DELIVERIES) AS TOTAL_DELIVERIES_SUM
FROM DEV.RAW_DATA.SIDO_DELIVERY_INFO
GROUP BY RECEIPT_DATE,DEPARTURE_AREA, DESTINATION_AREA;

*DEPARTURE_AREA_TOTAL_SUMMARY*


In [ ]:
--출발지역기준 전체 물동량
CREATE OR REPLACE TABLE ANALYTICS.DEPARTURE_AREA_TOTAL_SUMMARY AS
SELECT
    LEFT(RECEIPT_DATE,4) AS RECEIPT_YEAR,
    DEPARTURE_AREA,
    ISO_3166_2_CODE,
    SUM(TOTAL_DELIVERIES_SUM) AS DEPARTURE_AREA_TOTAL_DELIVERIES_SUM,
    RANK() OVER (ORDER BY DEPARTURE_AREA_TOTAL_DELIVERIES_SUM) AS ranking,
    LOG(10, SUM(TOTAL_DELIVERIES_SUM)) AS LOGARITHM_DEPARTURE_AREA_TOTAL_DELIVERIES_SUM
FROM ANALYTICS.AREA_TOTAL_SUMMARY AS A
JOIN ISO_3166_2_KR AS B ON A.DEPARTURE_AREA = B.AREA_NAME
GROUP BY DEPARTURE_AREA, ISO_3166_2_CODE, RECEIPT_YEAR;
--테이블에서 실제값, 랭킹, 로그값 중 골라서 사용가능

DESTINATION_AREA_TOTAL_SUMMARY

In [ ]:
--도착지역기준 전체 물동량
CREATE OR REPLACE TABLE ANALYTICS.DESTINATION_AREA_TOTAL_SUMMARY AS
SELECT
    LEFT(RECEIPT_DATE,4) AS RECEIPT_YEAR,
    DESTINATION_AREA,
    ISO_3166_2_CODE,
    SUM(TOTAL_DELIVERIES_SUM) AS DESTINATION_AREA_TOTAL_DELIVERIES_SUM,
    RANK() OVER (ORDER BY DESTINATION_AREA_TOTAL_DELIVERIES_SUM) AS ranking,
    LOG(10, SUM(TOTAL_DELIVERIES_SUM)) AS LOGARITHM_ESTINATION_AREA_TOTAL_DELIVERIES_SUM
FROM ANALYTICS.AREA_TOTAL_SUMMARY AS A
JOIN ISO_3166_2_KR AS B ON A.DESTINATION_AREA = B.AREA_NAME
GROUP BY DESTINATION_AREA, ISO_3166_2_CODE, RECEIPT_YEAR;
--테이블에서 실제값, 랭킹, 로그값 중 골라서 사용가능

### 카테고리별 물동량

####    (연도/월)별 카테고리 건수

sido_category_summary_groupby_month

In [ ]:
CREATE OR REPLACE TABLE SIDO_CATEGORY_SUMMARY_GROUPBY_MONTH AS
SELECT SUBSTR(RECEIPT_DATE, 6, 2) AS month, CATEGORY, SUM(TOTAL_DELIVERIES) AS total_deliveries
FROM dev.raw_data.Sido_delivery_info
WHERE RECEIPT_DATE IS NOT NULL
GROUP BY month, CATEGORY
ORDER BY 1,2;

sido_category_summary_groupby_year

In [ ]:
CREATE OR REPLACE TABLE SIDO_CATEGORY_SUMMARY_GROUPBY_YEAR AS
SELECT EXTRACT(YEAR FROM RECEIPT_DATE) AS year, DESTINATION_AREA, CATEGORY, SUM(TOTAL_DELIVERIES) AS total_deliveries
FROM
    dev.raw_data.Sido_delivery_info
WHERE RECEIPT_DATE IS NOT NULL
GROUP BY year, DESTINATION_AREA, CATEGORY;

#### 지역별 카테고리 건수

sido_category_summary

In [ ]:
CREATE OR REPLACE TABLE SIDO_CATEGORY_SUMMARY AS
SELECT DESTINATION_AREA, CATEGORY, SUM(TOTAL_DELIVERIES) AS total_deliveries
FROM
    dev.raw_data.Sido_delivery_info
WHERE RECEIPT_DATE IS NOT NULL
GROUP BY DESTINATION_AREA, CATEGORY;

#### (연도)지역별 생활물류 카테고리 건수

seoul_life_logistics_category_all

In [ ]:
CREATE OR REPLACE TABLE dev.analytics.SEOUL_LIFE_LOGISTICS_CATEGORY_ALL as
SELECT *
FROM (
SELECT
    recipient_district,
    '가구/인테리어' AS Category,
    SUM(category_furniture_and_interior) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '기타' AS Category,
    SUM(category_others) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '도서/음반' AS Category,
    SUM(category_book_and_music) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '디지털/가전' AS Category,
    SUM(category_digital_and_appliances) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '생활/건강' AS Category,
    SUM(category_living_and_health) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '스포츠/레저' AS Category,
    SUM(category_sports_and_leisure) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '식품' AS Category,
    SUM(category_food) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '출산/육아' AS Category,
    SUM(category_parenting) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '패션의류' AS Category,
    SUM(category_fashion_clothing) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '패션잡화' AS Category,
    SUM(category_fashion_accessories) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1
UNION
SELECT
    recipient_district,
    '화장품/미용' AS Category,
    SUM(category_beauty_cosmetics) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1)
ORDER BY recipient_district;

seoul_life_logistics_category_group_by_year

In [ ]:
CREATE OR REPLACE TABLE dev.analytics.SEOUL_LIFE_LOGISTICS_CATEGORY_GROUP_BY_YEAR as
SELECT *
FROM (
-- 카테고리 바꾸기
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '가구/인테리어' AS Category,
    SUM(category_furniture_and_interior) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '기타' AS Category,
    SUM(category_others) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '도서/음반' AS Category,
    SUM(category_book_and_music) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '디지털/가전' AS Category,
    SUM(category_digital_and_appliances) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '생활/건강' AS Category,
    SUM(category_living_and_health) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '스포츠/레저' AS Category,
    SUM(category_sports_and_leisure) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '식품' AS Category,
    SUM(category_food) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '출산/육아' AS Category,
    SUM(category_parenting) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '패션의류' AS Category,
    SUM(category_fashion_clothing) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '패션잡화' AS Category,
    SUM(category_fashion_accessories) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2
UNION
SELECT
    YEAR(delivery_date) AS year,
    recipient_district,
    '화장품/미용' AS Category,
    SUM(category_beauty_cosmetics) AS Value
FROM raw_data.Seoul_life_logistics
WHERE recipient_city = '서울특별시' AND recipient_district!='NaN'
GROUP BY 1, 2)
ORDER BY year, recipient_district;

### 기간별 물동량 추이

DAY_TOTAL_SUMMARY

In [ ]:
--DAY_TOTAL_SUMMARY 일별 총물동량
CREATE OR REPLACE TABLE ANALYTICS.DAY_TOTAL_SUMMARY AS
SELECT
    RECEIPT_DATE AS DAY_DATE,
    SUM(TOTAL_DELIVERIES) AS TOTAL_DELIVERIES_SUM
FROM DEV.RAW_DATA.SIDO_DELIVERY_INFO
GROUP BY DAY_DATE;

### 지역별 배송기간

CITY_DELIVINFO_DISTANCES

In [ ]:
create or replace table city_delivinfo_distances as
select rc_src.sido_nm as src_city, src_region_code, rc_dest.sido_nm as dest_city, dest_region_code, deliv_days, deliv_counts, distance
    from region_delivinfo_distances rdd
        join raw_data.full_region_codes rc_src
        on rdd.src_region_code = rc_src.region_code
        join raw_data.full_region_codes rc_dest
        on rdd.dest_region_code = rc_dest.region_code;
select sum(deliv_counts) from city_delivinfo_distances;

create or replace table city_delivinfo_distances_isocode as
select src_iso.iso_3166_2_code as src_iso, dest_iso.iso_3166_2_code as dest_iso, cdd.*
    from city_delivinfo_distances cdd
    join public.iso_3166_2_kr src_iso
    on cdd.src_city = src_iso.area_name
    join public.iso_3166_2_kr dest_iso
    on cdd.dest_city = dest_iso.area_name;

COUNTY_DELIVINFO_DISTANCES

In [ ]:
create or replace table county_delivinfo_distances as
select concat(rc_src.sido_nm, ' ', rc_src.sigungu_nm) as src_county, src_region_code, concat(rc_dest.sido_nm, ' ', rc_dest.sigungu_nm) as dest_county, dest_region_code, deliv_days, deliv_counts, distance
    from region_delivinfo_distances rdd
        join raw_data.region_codes rc_src
        on rdd.src_region_code = rc_src.region_code
        join raw_data.region_codes rc_dest
        on rdd.dest_region_code = rc_dest.region_code;
select sum(deliv_counts) from county_delivinfo_distances;

DELIVERY_QUANTITY_ROUTE

In [ ]:
create or replace table delivery_quantity_route as
SELECT
    SRC_CITY, DEST_CITY,
    CONCAT(SRC_CITY, ' -> ', DEST_CITY) AS deliv_route,
    sum(deliv_days*deliv_counts)/sum(deliv_counts) as avg_deliv_days,
    sum("DELIV_COUNTS") AS deliv_counts
FROM analytics.city_delivinfo_distances
where src_city not like '제주%' and dest_city not like '제주%'
GROUP BY 1,2
order by 5 desc;

DELIVERY_HARSHREGION_ROUTE

In [ ]:
create table delivery_harshregion_route as
SELECT
    SRC_CITY, DEST_CITY,
    CONCAT(SRC_CITY, ' -> ', DEST_CITY) AS deliv_route,
    sum(deliv_days*deliv_counts)/sum(deliv_counts) as avg_deliv_days,
    avg(distance) as avg_distance
FROM analytics.city_delivinfo_distances
where src_city like '제주%' or dest_city like '제주%'
GROUP BY 1,2
ORDER BY avg_deliv_days desc;

DELIVERY_CATEGORY_ROUTE

In [ ]:
create or replace table delivery_category_route as
select
    src_city,
    dest_city,
    deliv_route,
    deliv_counts,
    avg_deliv_days,
    avg_distance,
    case
        when (src_city like '제주%') or (dest_city like '제주%') then '산간지역 배송'
        when (RANK() OVER (ORDER BY deliv_counts DESC)) <= 10 then '물량 TOP 10 배송경로'
        when src_city like dest_city then '같은지역 배송'
        else '기타'
    end as route_category
from(
SELECT
    SRC_CITY, DEST_CITY,
    CONCAT(SRC_CITY, ' -> ', DEST_CITY) AS deliv_route,
    sum(deliv_days*deliv_counts)/sum(deliv_counts) as avg_deliv_days,
    sum("DELIV_COUNTS") AS deliv_counts,
    avg(distance) as avg_distance
FROM analytics.city_delivinfo_distances
GROUP BY 1,2
)
order by deliv_counts desc;

### 서울시 지역별 연령대별 통계

SEOUL_TOP1CATEGORY_GROUPBY_AREA

In [ ]:
create or replace table analytics.seoul_top1category_groupby_area as
select year, recipient_district, category, value
from (
    select *, 
    RANK() OVER (PARTITION BY year, recipient_district 
    order by value desc) as rank
    from analytics.seoul_life_logistics_category_group_by_year
)
where rank = 1
order by year;

SEOUL_TOP1_AGEGROUP

In [ ]:
create or replace table analytics.seoul_top1_agegroup_2020 as
    WITH RankedAges AS (
        SELECT 
            AREA_NAME, 
            AGE_GROUP, 
            TOTAL_POPULATION,
            ROW_NUMBER() OVER (PARTITION BY AREA_NAME ORDER BY TOTAL_POPULATION DESC) as rank
        FROM 
            raw_data.population_data_2020
        WHERE
            AGE_GROUP != '계' AND
            area_name in ( 
                select distinct recipient_district
                from analytics.seoul_top1category_groupby_area)
    )
    SELECT 
        AREA_NAME, 
        AGE_GROUP, 
        TOTAL_POPULATION
    FROM 
        RankedAges
    WHERE 
        rank = 1;

ALTER TABLE analytics.seoul_top1_agegroup_2020
ADD COLUMN YEAR INT;

UPDATE analytics.seoul_top1_agegroup_2020
SET YEAR = 2020;

        
create or replace table analytics.seoul_top1_agegroup_2021 as
    WITH RankedAges AS (
        SELECT 
            AREA_NAME, 
            AGE_GROUP, 
            TOTAL_POPULATION,
            ROW_NUMBER() OVER (PARTITION BY AREA_NAME ORDER BY TOTAL_POPULATION DESC) as rank
        FROM 
            raw_data.population_data_2021
        WHERE
            AGE_GROUP != '계' AND
            area_name in ( 
                select distinct recipient_district
                from analytics.seoul_top1category_groupby_area)
    )
    SELECT 
        AREA_NAME, 
        AGE_GROUP, 
        TOTAL_POPULATION
    FROM 
        RankedAges
    WHERE 
        rank = 1;

ALTER TABLE analytics.seoul_top1_agegroup_2021
ADD COLUMN YEAR INT;

UPDATE analytics.seoul_top1_agegroup_2021
SET YEAR = 2021;

        
create or replace table analytics.seoul_top1_agegroup_2022 as
    WITH RankedAges AS (
        SELECT 
            AREA_NAME, 
            AGE_GROUP, 
            TOTAL_POPULATION,
            ROW_NUMBER() OVER (PARTITION BY AREA_NAME ORDER BY TOTAL_POPULATION DESC) as rank
        FROM 
            raw_data.population_data_2022
        WHERE
            AGE_GROUP != '계' AND
            area_name in ( 
                select distinct recipient_district
                from analytics.seoul_top1category_groupby_area)
    )
    SELECT 
        AREA_NAME, 
        AGE_GROUP, 
        TOTAL_POPULATION
    FROM 
        RankedAges
    WHERE 
        rank = 1;

ALTER TABLE analytics.seoul_top1_agegroup_2022
ADD COLUMN YEAR INT;

UPDATE analytics.seoul_top1_agegroup_2022
SET YEAR = 2022;

SEOUL_AGE_WITH_CATEGORY  
서울시 지역별 1위 인구수와 1위 카테고리

In [ ]:
create or replace table analytics.seoul_age_with_category_2021 as
SELECT
    a.area_name,
    a.age_group,
    a.total_population,
    a. year,
    c.category,
    c.value
FROM
    analytics.seoul_top1_agegroup_2021 a
INNER JOIN
    analytics.seoul_top1category_groupby_area c
ON
    a.year = c.year AND
    a.area_name = c.recipient_district;


    
create or replace table analytics.seoul_age_with_category_2022 as
SELECT
    a.area_name,
    a.age_group,
    a.total_population,
    a. year,
    c.category,
    c.value
FROM
    analytics.seoul_top1_agegroup_2022 a
INNER JOIN
    analytics.seoul_top1category_groupby_area c
ON
    a.year = c.year AND
    a.area_name = c.recipient_district;